In [4]:
import ee
import rasterio
import numpy as np
from pprint import pprint
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import colors

In [10]:
from channel_classifier.internal.ee_utils import compute_error_image, download_tiff, get_landsat_image, form_labelled_image, ndvi_timeseries

In [11]:
ee.Initialize()

In [12]:
MODIS_LABELS = ['']

In [13]:
path, row = [7, 65] # Ucayali
year, month, day = [2014, 9, 7]
img = get_landsat_image(path, row, year, month, day)

In [14]:
start_date = '2014-01-01'
end_date = '2018-01-01'
dataset_path = 'LANDSAT/LC08/C01/T1_TOA'
collection = ee.ImageCollection(dataset_path) \
    .filter(ee.Filter.eq('WRS_PATH', path)) \
    .filter(ee.Filter.eq('WRS_ROW', row)) \
    .filterDate(start_date, end_date)
ndvis = ndvi_timeseries(collection)

TypeError: map() got an unexpected keyword argument 'dropNulls'

In [15]:
import matplotlib
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')
matplotlib.rcParams['animation.writer'] = 'avconv'
def download_images(images):
    ims = []
    images_list = images.toList(images.size().getInfo())
    for i in range(images_list.size().getInfo()):
        image = images_list.get(i)
        filename = download_tiff(image)
        with rasterio.open(filename) as r:
            im = r.read(1)
        ims.append(im)
    return ims
  
def animate_images(images):
    fig = plt.figure(figsize=(12, 12))
    ims = [[plt.imshow(x, cmap='BrBG', animated=True)] for x in images]
    anim = animation.ArtistAnimation(fig, ims, interval=60, blit=True, repeat_delay=1000)
    #anim.save('fig.mp4')
    HTML(anim.to_html5_video())

In [ ]:
ims = download_images(ndvis)

In [ ]:
animate_images(ims)

In [ ]:
vis_params = {'bands' : 'B5, B6, B4'} # XXX: bands should be a string, not list...
url = img.getThumbUrl(vis_params)
print(ee.Date(img.get('system:time_start')).format('Y-M-d').getInfo())

In [ ]:
Image(url=url)

In [ ]:
used_bands = ['B{}'.format(x) for x in np.arange(1, 8)]
stacked = form_labelled_image(path, row, year, month, day, bands=used_bands)

In [ ]:
filename = download_tiff(stacked, scale=200)

In [ ]:
cmap = colors.ListedColormap(['white', 'blue', 'red'])
bounds = [0, 0.5, 1.5, 2]
norm = colors.BoundaryNorm(bounds, cmap.N)
with rasterio.open(filename) as r:
    i = r.count
    labels = r.read(i)
    labels[labels == 0] = np.nan
    label_values = list(np.unique(labels[~np.isnan(labels)]))
fig, ax = plt.subplots(1, 1, figsize=(12,12))
im = ax.imshow(labels, cmap='Set1', vmin=0)#, cmap=cmap, norm=norm)
cb = plt.colorbar(im, ax=ax, shrink=0.5)#, label='Class label', ticks=[0, 1, 2])
#cb.ax.set_yticklabels(label_values);

In [ ]:
bands = stacked.bandNames().getInfo()
print(bands)

In [ ]:
training = stacked.sample(numPixels=10000, seed=0).select(bands).filter(ee.Filter.neq('label', 0))
validation = stacked.sample(numPixels=10000, seed=1).select(bands).filter(ee.Filter.neq('label', 0))

In [ ]:
classifier = ee.Classifier.randomForest(10).train(training, 'label')

In [ ]:
classified = stacked.classify(classifier, 'classification')

In [ ]:
validated = validation.classify(classifier, 'classification')
accuracy = validated.errorMatrix('label', 'classification')
print(accuracy.accuracy().getInfo())

In [ ]:
error = compute_error_image(clipped_labels, classified.clip(img.geometry()), 1)

In [ ]:
filename = download_tiff(error, scale=240)
with rasterio.open(filename) as r:
    err = r.read(1)
fig, ax = plt.subplots(1, 1, figsize=(12,12))
im = ax.imshow(err, cmap='RdBu_r', vmin=-1, vmax=1) #, cmap=cmap, norm=norm)
cb = plt.colorbar(im, ax=ax, shrink=0.5, ticks=[-1, 0, 1])
cb.ax.set_yticklabels(['Omission', '', 'Comission']);

In [ ]:
filename = download_tiff(classified, scale=200)

In [ ]:
cmap = colors.ListedColormap(['white', 'blue', 'red'])
bounds = [0, 0.5, 1.5, 2]
norm = colors.BoundaryNorm(bounds, cmap.N)
with rasterio.open(filename) as r:
    i = r.count
    labels = r.read(i).astype(float)
    labels[labels == 0] = np.nan
fig, ax = plt.subplots(1, 1, figsize=(12,12))
im = ax.imshow(labels, cmap='Set1')#, cmap=cmap, norm=norm)
cb = plt.colorbar(im, ax=ax, shrink=0.5)#, label='Class label', ticks=[0, 1, 2])
cb.ax.set_yticklabels(np.arange(1, 17));